In [1]:
%pip install torchinfo

  Using cached torchinfo-1.8.0-py3-none-any.whl.metadata (21 kB)
Using cached torchinfo-1.8.0-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


책에는 없는 듯하다...

#### PyTorch를 이용해보자!
- PyTorch = Tensor + Autograd + Neural Network

| 기능 영역       | 예시                                                            |
| -------------- | -------------------------------------------------------------- |
| **Tensor 연산** | `torch.tensor()`, `torch.matmul()`, `torch.sum()` 등           |
| **자동 미분**    | `tensor.requires_grad = True`, `loss.backward()`              |
| **신경망 구성**  | `torch.nn.Linear`, `torch.nn.Sequential`, `torch.nn.Module` 등 |
| **옵티마이저**   | `torch.optim.SGD`, `Adam`, `RMSprop`                           |
| **GPU 연산**    | `tensor.to('cuda')` 또는 `.cuda()`                              |

In [1]:
import torch
from torchvision.datasets import FashionMNIST
fm_train = FashionMNIST(root='.', train=True, download=True)  # 훈련세트
fm_test = FashionMNIST(root='.', train=False, download=True)  # 테스트세트

100%|██████████| 26.4M/26.4M [00:04<00:00, 6.35MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 112kB/s]
100%|██████████| 4.42M/4.42M [00:02<00:00, 2.03MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 3.62MB/s]


FashionMNIST를 해당 파일 경로에 저장했다.

In [3]:
type(fm_train.data)

torch.Tensor

In [4]:
# 훈련세트, 테스트세트 개수
print(fm_train.data.shape, fm_test.data.shape)
print(fm_train.targets.shape, fm_test.targets.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.Size([60000]) torch.Size([10000])


In [6]:
train_input = fm_train.data
train_target = fm_train.targets

In [7]:
# 정규화 (0~1)
train_scaled = train_input / 255.0

In [8]:
# 훈련세트, 검증세트 분리
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 층 구성 (Flatten 포함)
import torch.nn as nn  # nn: neural network (신경망 모듈)

model = nn.Sequential(
    nn.Flatten(),        # 2D 이미지(28*28)를 1D 벡터(784차원)로 펼침
    nn.Linear(784, 100), # 입력층: 노드 28*28=784개 → 은닉층: 유닛 100개
    nn.ReLU(),           # 활성화 함수: ReLU
    nn.Linear(100, 10)   # 은닉층: 유닛 100개 → 출력층: 클래스 10개
)

# 층 구성 요약 정보 - input_size=(배치_사이즈, ...) (다만, 이전과 달리 배치 사이즈 기본값은 없다)
from torchinfo import summary
summary(model, input_size=(32, 28, 28))  # 배치 사이즈를 32로 지정

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 10]                  --
├─Flatten: 1-1                           [32, 784]                 --
├─Linear: 1-2                            [32, 100]                 78,500
├─ReLU: 1-3                              [32, 100]                 --
├─Linear: 1-4                            [32, 10]                  1,010
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.54
Input size (MB): 0.10
Forward/backward pass size (MB): 0.03
Params size (MB): 0.32
Estimated Total Size (MB): 0.45

- 맥의 경우 - mps / 그래픽 가속기
```python
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
```

In [11]:
# GPU 또는 CPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # 모델에 GPU 또는 CPU 연산 설정

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=100, bias=True)
  (2): ReLU()
  (3): Linear(in_features=100, out_features=10, bias=True)
)

- 활성화 함수 설정
    - 이진 분류: nn.BCELoss() ->  손실함수 + 출력함수(sigmoid)
    - 다중 분류: nn.CrossEntropyLoss() -> 손실함수 + 출력함수(softmax)

In [13]:
# 손실 함수(loss), 옵티마이저(optimizer) 설정
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # 손실함수 - Cattegorical Crossentropy
optimizer = optim.Adam(model.parameters())  # 2번째 매개변수: learning_rate=? - 기본값 0.1

In [14]:
# 학습 (fit은 없다. 하나하나 코드를 짜야 함.)

epochs = 5
batches = int(len(train_scaled) / 32)  # 1500 (1 epoch당 1500번 학습)
for epoch in range(epochs):
    model.train()   # 학습 시작을 알려줌
    train_loss = 0  # loss값의 합
    for i in range(batches):  # 미니배치 경사 하강법
        # 1번째 epoch: 0 ~ 31, 2번째 epoch: 32 ~ 63, ... 1500번째 epoch: ~ len(train_scaled) - 1
        inputs = train_scaled[i * 32 : (i + 1) * 32].to(device)
        targets = train_target[i * 32 : (i + 1) * 32].to(device)

        optimizer.zero_grad()    # 그래디언트 초기화
        outputs = model(inputs)  # 추론

        loss = criterion(outputs, targets)  # loss(손실)값 구함
        loss.backward()   # 역전파
        optimizer.step()  # 모델 파라미터(가중치, 절편) 업데이트
        train_loss += loss.item()  # 현재 배치의 loss를 더함
    print(f"epochs: {epoch + 1}, loss: {train_loss / batches:.4f}")

epochs: 1, loss: 0.5466
epochs: 2, loss: 0.4030
epochs: 3, loss: 0.3614
epochs: 4, loss: 0.3340
epochs: 5, loss: 0.3125


In [24]:
model.eval()  # 모델 평가
with torch.no_grad():  # 평가 시에는 그래디언트 연산 X
    val_scaled = val_scaled.to(device)
    val_target = val_target.to(device)
    outputs = model(val_scaled)
    # print(outputs[:10])

    # argmax로 학습 결과 배열 가져오기
    predicts = torch.argmax(outputs, 1)
    print(predicts[:10])

    # 정답 개수 구하기
    corrects = (predicts == val_target).sum().item()  # True는 1로 연산, 정답 개수

    # 정확도, 손실값 출력
    accuracy = corrects / len(val_target)
    loss = criterion(outputs, val_target)
    print(f"accuracy:{accuracy:.4f}, loss:{loss:.4f}")

tensor([8, 8, 7, 4, 8, 4, 1, 0, 4, 5])
accuracy:0.8771, loss:0.3431
